# Intro & Loading 

In [ ]:
from __future__ import print_function
import pandas as pd

from genepy.utils import helper as h
from gsheets import Sheets
from taigapy import TaigaClient

%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
tc = TaigaClient()
output_notebook()

my_id = '~/.client_secret.json'
mystorage_id = "~/.storage.json"
sheets = Sheets.from_files(my_id, mystorage_id)
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

In [ ]:
samplesetname = "21Q1"

virtual= {}
virtual['public'] = 'public-21q1-4b39'
virtual['ibm'] = 'ibm-21q1-abd9'
virtual['dmc'] = 'dmc-21q1-0e11'
virtual['internal']='internal-21q1-4fc4'


taiga_mutation = {}
taiga_mutation['internal'] = "depmap-mutation-calls-9be3"
taiga_mutation['ibm'] = "mutations-b05c"
taiga_mutation['dmc'] = "depmap-mutation-calls-dfce"
taiga_mutation['public'] ="depmap-mutation-calls-9a1a"

taiga_expression = {}
taiga_expression['internal'] ="depmap-rnaseq-expression-data-363a"
taiga_expression['ibm'] = "expression-9d97"
taiga_expression['dmc'] = "depmap-rnaseq-expression-data-80ef"
taiga_expression['public'] ="depmap-rnaseq-expression-data-ccd0"

taiga_fusion = {}
taiga_fusion['internal'] = "gene-fusions-8b7a"
taiga_fusion['ibm'] = "fusion-dc08"
taiga_fusion['dmc'] = "gene-fusions-375f"
taiga_fusion['public'] ="gene-fusions-6212"

taiga_copynumber = {}
taiga_copynumber['internal'] ="depmap-wes-cn-data-81a7"
taiga_copynumber['ibm'] = "cn-e20f"
taiga_copynumber['dmc'] = "depmap-cn-data-9b9d"
taiga_copynumber['public'] = "depmap-wes-cn-data-97cc"

prev_virtual = {}
#prev_virtual['public'] = 'public-20q3-3d35'
#prev_virtual['dmc'] = 'dmc-20q3-deprecated-never-released--5f55'
#prev_virtual['internal'] = 'internal-20q3-00d0'

prev_virtual['public'] = 'public-20q4-a4b3'
prev_virtual['dmc'] = 'dmc-20q4-fcf4'
# prev_virtual['ibm'] = 'ibm-20q4-269f'
prev_virtual['internal'] = 'internal-20q4-2540'
potential_list_url = "https://docs.google.com/spreadsheets/d/1YuKEgZ1pFKRYzydvncQt9Y_BKToPlHP-oDB-0CAv3gE"

release = samplesetname.lower()

In [ ]:
new = {}
gsheets = sheets.get(potential_list_url).sheets[0].to_frame()
new['internal'] = set([i for i in gsheets['Internal'].values.tolist() if str(i) != "nan"])
new['dmc'] = set([i for i in gsheets['DMC'].values.tolist() if str(i) != "nan"])
new['ibm'] = set([i for i in gsheets['IBM'].values.tolist() if str(i) != "nan"])
new['public'] = set([i for i in gsheets['Public'].values.tolist() if str(i) != "nan"])


new["internal"] = new["internal"] | new["ibm"] | new["dmc"] | new["public"]
new["ibm"] = new["ibm"] | new["dmc"] | new["public"]
new["dmc"] = new["dmc"] | new["public"]

## Getting what was released before

In [ ]:
prevmut = {}
prevrna = {}
prevcn = {}
prevwes = {}
prev = {}
for val in ['internal', 'dmc', 'public']:
    print(val)
    prevmut[val] = set(tc.get(name=prev_virtual[val], file='CCLE_mutations').DepMap_ID)
    prevrna[val] = set(tc.get(name=prev_virtual[val], file='CCLE_expression').index)
    prevcn[val] = set(tc.get(name=prev_virtual[val], file='CCLE_segment_cn').DepMap_ID)
    prev[val] = prevmut[val] | prevrna[val] | prevcn[val]
    prevwes[val] = prevmut[val] | prevcn[val]

In [ ]:
prevmut["dmc"] = prevmut["dmc"] | prevmut["public"]
prevrna["dmc"] = prevrna["dmc"] | prevrna["public"]
prevcn["dmc"] = prevcn["dmc"] | prevcn["public"]
prev["dmc"] = prev["dmc"] | prev["public"]
prevwes["dmc"] = prevwes["dmc"] | prevwes["public"]

prevmut["internal"] = prevmut["internal"] | prevmut["dmc"]
prevrna["internal"] = prevrna["internal"] | prevrna["dmc"]
prevcn["internal"] = prevcn["internal"] | prevcn["dmc"]
prev["internal"] = prev["internal"] | prev["dmc"]
prevwes["internal"] = prevwes["internal"] | prevwes["dmc"]

In [ ]:
print('in cn but not mut')
print(prevwes["internal"] - prevmut["internal"])
print('in mut but not cn')
print(prevwes["internal"] - prevcn["internal"])
print('in rna but no wes/wgs')
print(prev["internal"] - prevwes["internal"])
print('in wes/wgs but not rna')
print(prev["internal"] - prevrna["internal"])

In [ ]:
prev['ibm'] = prev['dmc']
prevcn['ibm'] = prevcn['dmc']
prevmut['ibm'] = prevmut['dmc']
prevrna['ibm'] = prevrna['dmc']
prevwes['ibm'] = prevwes['dmc']

## managing the readmes

In [ ]:
#! cd .. && git clone https://github.com/broadinstitute/depmap-release-readmes.git && cd -

In [ ]:
! cd ../depmap-release-readmes && git pull

In [ ]:
!cd ../depmap-release-readmes/ && python3 make_new_release.py $release && git add . && git commit -m $release && git push 

In [ ]:
# NOW UPDATE THE READMEs

In [ ]:
! mkdir temp/README/

In [ ]:
! cd ../depmap-release-readmes && cp release-$release/* ../ccle_processing/readmes/ && git add . && git commit -m "Omics: updating readmes to new release" && git push

In [ ]:
! cd ../depmap-release-readmes && git pull &&  cp release-$release/* ../ccle_processing/readmes/

In [ ]:
for val in ['internal', 'ibm', 'dmc', 'public']:
    os.popen('cp readmes/'+val+'-'+release+'.txt temp/README').read()
    AddToVirtual(virtual[val], files = [('Raw','temp/README')])

# Mutations

## Somatic

In [ ]:
mutations = pd.read_csv("temp/wes_somatic_mutations_withlegacy_"+samplesetname+".csv")
damaging = pd.read_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_damaging.csv', index_col=0)
othercons = pd.read_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_othercons.csv', index_col=0)
othernoncons = pd.read_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_othernoncons.csv', index_col=0)
hotspot = pd.read_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_hotspot.csv', index_col=0)
hotspot=hotspot.astype(int)
damaging=damaging.astype(int)
othercons=othercons.astype(int)
othernoncons=othernoncons.astype(int)

In [ ]:
#reverting to previous versions
mutations = mutations[mutations.is_likely_immortalization!=True]
mutations = mutations[['Hugo_Symbol', 'Entrez_Gene_Id', 'NCBI_Build', 'Chromosome',
       'Start_position', 'End_position', 'Strand', 'Variant_Classification',
       'Variant_Type', 'Reference_Allele', 'Tumor_Allele', 'dbSNP_RS',
       'dbSNP_Val_Status', 'Genome_Change', 'Annotation_Transcript',
       'DepMap_ID', 'cDNA_Change', 'Codon_Change', 'Protein_Change', 'isDeleterious',
       'isTCGAhotspot', 'TCGAhsCnt', 'isCOSMIChotspot', 'COSMIChsCnt',
       'ExAC_AF',"Variant_annotation", 'CGA_WES_AC', 'HC_AC',
       'RD_AC', 'RNAseq_AC', 'SangerWES_AC', 'WGS_AC']].rename(columns={"Tumor_Allele":"Tumor_Seq_Allele1"})

In [ ]:
! ls temp/README

In [ ]:
"ACH-001502" in prevmut["ibm"]

In [ ]:
blacklist = set()
for val in ['internal', 'ibm', "dmc", "public"]:
    print('_________________________________________________')
    print(val)
    missing = set(new[val]) - set(mutations.DepMap_ID)
    print('nott present')
    removed = set(prev[val]) - set(mutations.DepMap_ID)
    print(removed)
    print('removed')
    removed = set(prevmut[val]) - set(mutations.DepMap_ID)
    print(removed)
    blacklist = (set(mutations.DepMap_ID) - (prevmut[val] | set(new[val]))) | blacklist
    print('missing')
    print(missing)
    newlines = set(new[val]) 
    print('blacklist')
    print(blacklist)
    # adding files
    a = mutations[~mutations.DepMap_ID.isin(blacklist)]
    print(len(mutations)- len(a))
    a.to_csv('temp/all_somatic_mutations_withlegacy.csv', index=False)
    a = damaging[~damaging.index.isin(blacklist)]
    print(len(damaging) - len(a))
    a.to_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_damaging.csv')
    a = othercons[~othercons.index.isin(blacklist)]
    print(len(othercons) - len(a))
    a.to_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_othercons.csv')
    a = othernoncons[~othernoncons.index.isin(blacklist)]
    print(len(othernoncons) - len(a))
    a.to_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_othernoncons.csv')
    a = hotspot[~hotspot.index.isin(blacklist)]
    print(len(hotspot) - len(a))
    a.to_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_hotspot.csv')
    os.popen('cp readmes/'+val+'-'+release+'.txt temp/README').read()
    
    # updating on taiga
    tc.update_dataset(dataset_permaname=taiga_mutation[val],
                     upload_file_path_dict={
    'temp/all_somatic_mutations_withlegacy.csv': 'TableCSV',
    'temp/all_somatic_mutations_boolmatrix_fordepmap_damaging.csv': 'NumericMatrixCSV',
    'temp/all_somatic_mutations_boolmatrix_fordepmap_othernoncons.csv': 'NumericMatrixCSV',
    'temp/all_somatic_mutations_boolmatrix_fordepmap_othercons.csv': 'NumericMatrixCSV',
    'temp/all_somatic_mutations_boolmatrix_fordepmap_hotspot.csv': 'NumericMatrixCSV',
    'temp/README': 'Raw'},
                     dataset_description="""
    #Mutations Omics:

    for informations, see README

    NEW LINES:
    """+str(newlines)+"""

    BLACKLISTED:
    """+str(blacklist))

    # To add to a virtual dataset
    AddToVirtual(virtual[val], taiga_mutation[val], [('CCLE_mutations', 'all_somatic_mutations_withlegacy'),
    ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_damaging', 'all_somatic_mutations_boolmatrix_fordepmap_damaging'),
    ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_othernoncons', 'all_somatic_mutations_boolmatrix_fordepmap_othernoncons'),
    ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_othercons', 'all_somatic_mutations_boolmatrix_fordepmap_othercons'),
    ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_hotspot', 'all_somatic_mutations_boolmatrix_fordepmap_hotspot'), ('README','README')])

In [ ]:
# To add to a eternal dataset
AddToVirtual('depmap-a0ab', taiga_mutation['internal'], [('CCLE_mutations', 'all_somatic_mutations_withlegacy'),
('CCLE_all_somatic_mutations_boolmatrix_fordepmap_damaging', 'all_somatic_mutations_boolmatrix_fordepmap_damaging'),
('CCLE_all_somatic_mutations_boolmatrix_fordepmap_othernoncons', 'all_somatic_mutations_boolmatrix_fordepmap_othernoncons'),
('CCLE_all_somatic_mutations_boolmatrix_fordepmap_othercons', 'all_somatic_mutations_boolmatrix_fordepmap_othercons'),
('CCLE_all_somatic_mutations_boolmatrix_fordepmap_hotspot', 'all_somatic_mutations_boolmatrix_fordepmap_hotspot')])

# Copy Number

In [ ]:
genecn= pd.read_csv('temp/all_'+samplesetname+'_gene_cn.csv',index_col=0)
segmentcn = pd.read_csv('temp/all_'+samplesetname+'_segment.csv')

In [ ]:
#genecn = genecn.apply(lambda x: (x**2)-1)

In [ ]:
blacklist = set()
for val in ['internal', 'ibm', 'dmc', 'public']:
    print('_________________________________________________')
    print(val)
    print('not present')
    removed = set(prevcn[val]) - set(segmentcn.DepMap_ID)
    print(removed)
    print('removed')
    removed = set(prevcn[val]) - set(segmentcn.DepMap_ID)
    print(removed)
    missing = set(new[val]) - set(segmentcn.DepMap_ID)
    blacklist = (set(segmentcn.DepMap_ID) - (prevcn[val] | set(new[val]))) | blacklist
    print('missing')
    print(missing)
    newlines = set(new[val]) 
    print('blacklist')
    print(len(blacklist), blacklist)
    ## for segment removing first blacklisted, then embargoed, to create two datasets
    print(len(segmentcn))
    a = segmentcn[~segmentcn.DepMap_ID.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/all_merged_segments.csv', index=False)
    print(len(genecn))
    a = genecn[~genecn.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/all_merged_genes_cn.csv')
    
    os.popen('cp readmes/'+val+'-'+release+'.txt temp/README')

    # Add to Taiga
    tc.update_dataset(dataset_permaname=taiga_copynumber[val], 
                      upload_file_path_dict={
                        'temp/all_merged_genes_cn.csv': 'NumericMatrixCSV',
                        'temp/all_merged_segments.csv': 'TableCSV',
                        'temp/README': 'Raw'},
                      dataset_description=
    """
    #Copy Number Omics:

    for informations, see README

    NEW LINES:
    """+str(newlines)+"""

    BLACKLIST:
    """+str(blacklist))
    # To add to a virtual dataset
    AddToVirtual(virtual[val], taiga_copynumber[val], [('CCLE_gene_cn', 'all_merged_genes_cn'), ('CCLE_segment_cn','all_merged_segments')])

In [ ]:
# To add to a eternal dataset
AddToVirtual('depmap-a0ab', taiga_copynumber['internal'], [('CCLE_gene_cn', 'all_merged_genes_cn'), ('CCLE_segment_cn', 'all_merged_segments')])

In [ ]:
ls

# RNA

In [ ]:
transcripts_tpm = pd.read_csv('temp/expression_' + samplesetname + '_transcripts_tpm.csv',index_col=0)
genes_tpm = pd.read_csv('temp/expression_' + samplesetname + '_genes_tpm.csv',index_col=0)
genes_expected_count = pd.read_csv('temp/expression_' + samplesetname + '_genes_expected_count.csv',index_col=0)
proteincoding_genes_expected_count = pd.read_csv('temp/expression_' + samplesetname + '_proteincoding_genes_expected_count.csv',index_col=0)
proteincoding_genes_tpm = pd.read_csv('temp/expression_' + samplesetname + '_proteincoding_genes_tpm.csv',index_col=0)
transcripts_expected_count = pd.read_csv('temp/expression_' + samplesetname + '_transcripts_expected_count.csv',index_col=0)

In [ ]:
#putting it back to what it was before

# log transforming tpm data
transcripts_tpm=transcripts_tpm.apply(lambda x: np.log2(x+1))
genes_tpm=genes_tpm.apply(lambda x: np.log2(x+1))
proteincoding_genes_tpm=proteincoding_genes_tpm.apply(lambda x: np.log2(x+1))

In [ ]:
%store -r failed

In [ ]:
blacklist = set()
for val in ['internal','ibm', 'dmc','public']:
    print('_________________________________________________')
    print(val)
    print('not present')
    removed = set(prev[val]) - set(genes_tpm.index)
    print(removed)
    print('removed for QC reasons')
    print(failed)
    print('removed')
    removed = set(prevrna[val]) - set(genes_tpm.index)
    #print(removed - set(rename.keys()))
    missing = set(new[val]) - set(genes_tpm.index)
    blacklist = (set(genes_tpm.index) - (prevrna[val] | set(new[val]))) | blacklist
    print('missing')
    print(missing)
    newlines = set(new[val]) 
    print('blacklist')
    print(len(blacklist), blacklist)

    ## removing first blacklisted, then embargoed, to create two datasets
    print(len(genes_expected_count))
    a = genes_expected_count[~genes_expected_count.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/expression_genes_expected_count.csv')
    print(len(genes_tpm))
    a = genes_tpm[~genes_tpm.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/expression_genes_tpm.csv')
    print(len(proteincoding_genes_tpm))
    a = proteincoding_genes_tpm[~proteincoding_genes_tpm.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/expression_proteincoding_genes_tpm.csv')
    print(len(transcripts_tpm))
    a = transcripts_tpm[~transcripts_tpm.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/expression_transcripts_tpm.csv')
    print(len(proteincoding_genes_expected_count))
    a = proteincoding_genes_expected_count[~proteincoding_genes_expected_count.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/expression_proteincoding_genes_expected_count.csv')
    print(len(transcripts_expected_count))
    a = transcripts_expected_count[~transcripts_expected_count.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/expression_transcripts_expected_count.csv')
    
    os.popen('cp readmes/'+val+'-'+release+'.txt temp/README')

    # adding to taiga
    tc.update_dataset(dataset_permaname=taiga_expression[val],
                     upload_file_path_dict={
                       'temp/expression_genes_expected_count.csv': 'NumericMatrixCSV',
                       'temp/expression_transcripts_tpm.csv': 'NumericMatrixCSV',
                       'temp/expression_genes_tpm.csv': 'NumericMatrixCSV',
                       'temp/expression_proteincoding_genes_tpm.csv': 'NumericMatrixCSV',
                       'temp/expression_proteincoding_genes_expected_count.csv': 'NumericMatrixCSV',
                       'temp/expression_transcripts_expected_count.csv': 'NumericMatrixCSV',
                     "temp/README": "Raw"},
                      dataset_description=
    """
    # INTERNAL RNA

    for information, see README

    NEW LINES:
    """+str(newlines)+"""

    REMOVED FOR QC REASONS:
    """+str(failed)+"""

    BLACKLIST:
    """+str(blacklist))

    # add to virtual 
    AddToVirtual(virtual[val], taiga_expression[val], files=[
    ('CCLE_expression_full', 'expression_genes_tpm'), 
    ('CCLE_RNAseq_transcripts', 'expression_transcripts_tpm'),
    ('CCLE_RNAseq_reads', 'expression_genes_expected_count'),
    ('CCLE_expression', 'expression_proteincoding_genes_tpm'),
    ('CCLE_expression_proteincoding_genes_expected_count', 'expression_proteincoding_genes_expected_count'),
    ('CCLE_expression_transcripts_expected_count', 'expression_transcripts_expected_count')])

In [ ]:
AddToVirtual('depmap-a0ab', taiga_expression['internal'], files=[
('CCLE_expression_full', 'expression_genes_tpm'), 
('CCLE_RNAseq_transcripts', 'expression_transcripts_tpm'),
('CCLE_RNAseq_reads', 'expression_genes_expected_count'),
('CCLE_expression', 'expression_proteincoding_genes_tpm'), ('CCLE_expression_proteincoding_genes_expected_count', 'expression_proteincoding_genes_expected_count'),('CCLE_expression_transcripts_expected_count', 'expression_transcripts_expected_count')])

# Fusions

In [ ]:
fusions = pd.read_csv('temp/fusions_'+samplesetname+'.csv')
filtered = pd.read_csv('temp/filtered_fusions_'+samplesetname+'.csv')

In [ ]:
%store -r failed

In [ ]:
print(set(fusions.DepMap_ID) - set(filtered.DepMap_ID))
print(set(filtered.DepMap_ID) - set(fusions.DepMap_ID))

In [ ]:
blacklist = set()
for val in ['internal', 'ibm', 'dmc', 'public']:
    print('_________________________________________________')
    print(val)
    print('not present')
    removed = set(prev[val]) - set(fusions.DepMap_ID)
    print(removed)
    print('removed for QC reasons')
    print(failed)
    print('removed')
    removed = set(prevrna[val]) - set(fusions.DepMap_ID)
    print(removed - set(rename.keys()))
    missing = set(new[val]) - set(fusions.DepMap_ID)
    blacklist = (set(fusions.DepMap_ID) - (prevrna[val] | set(new[val]))) | blacklist
    print('missing')
    print(missing)
    newlines = set(new[val]) 
    print('blacklist')
    print(len(blacklist), blacklist)
    ## removing first blacklisted, then embargoed, to create two datasets
    print(len(fusions))
    a = fusions[~fusions.DepMap_ID.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/fusions.csv', index=False)
    print(len(filtered))
    a= filtered[~filtered.DepMap_ID.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/filtered_fusions.csv', index=False)

    os.popen('cp readmes/'+val+'-'+release+'.txt temp/README')

    # uploading to taiga
    tc.update_dataset(dataset_permaname=taiga_fusion[val],
                     upload_file_path_dict={
                         'temp/fusions.csv': 'TableCSV',
                         'temp/filtered_fusions.csv': 'TableCSV',
                         'temp/README': 'Raw'},
                      dataset_description=
    """
    # Internal Fusions

    for more information, see README

    NEW LINES:
    """+str(newlines)+"""

    REMOVED FOR QC REASONS:
    """+str(failed)+"""

    BLACKLIST:
    """+str(blacklist))

    # virtual datasets
    AddToVirtual(virtual[val], taiga_fusion[val], files=[('CCLE_fusions_unfiltered', 'fusions'),('CCLE_fusions', 'filtered_fusions')])

In [ ]:
AddToVirtual('depmap-a0ab', "gene-fusions-8b7a", files=[('CCLE_fusions_unfiltered', 'fusions'),('CCLE_fusions', 'filtered_fusions')])